In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, max_error
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score

def prepare_data_for_prediction(new_data):
    # Handle age ranges
    new_data['age_numeric'] = new_data['age'].apply(age_to_numeric)
    
    # Feature engineering
    new_data['sample_size_log'] = np.log1p(new_data['sample_size'])
    
    # Add safety likelihood feature
    new_data['safety_likelihood'] = new_data.apply(safety_likelihood, axis=1)
    
    # Select features
    X_new = new_data[numeric_features + categorical_features]
    
    return X_new

def make_predictions(model, new_data):
    X_new = prepare_data_for_prediction(new_data)
    predictions = model.predict(X_new)
    return predictions

def identify_high_risk_areas(data, threshold=0.7):
    data['risk_score'] = data['predicted_likelihood'] * (1 - data['safety_likelihood'])
    high_risk_areas = data[data['risk_score'] > threshold]
    return high_risk_areas.sort_values('risk_score', ascending=False)

# Load the data
data = pd.read_csv('HB1.csv')

# Filter data for smoking behavior only
data = data[data['behavior'] == 'Smoking']

# Identify numeric and categorical columns
numeric_columns = data.select_dtypes(include=[np.number]).columns
categorical_columns = data.select_dtypes(exclude=[np.number]).columns

# Handle age ranges
def age_to_numeric(age_range):
    if isinstance(age_range, str):
        if '-' in age_range:
            return int(age_range.split('-')[0])
        elif '+' in age_range:
            return int(age_range.replace('+', ''))
        else:
            return int(age_range)
    return age_range

data['age_numeric'] = data['age'].apply(age_to_numeric)

# Feature engineering
data['sample_size_log'] = np.log1p(data['sample_size'])

# Add safety likelihood feature
def safety_likelihood(row):
    base_safety = 0.8
    safety = base_safety - 0.1 if row['gender'] == 'Female' else base_safety
    safety -= 0.05 if 'CBD' in str(row['location']) or 'Melbourne' in str(row['location']) else 0
    safety -= 0.05 if row['age_numeric'] < 25 or row['age_numeric'] > 65 else 0
    return max(0, min(safety, 1))

data['safety_likelihood'] = data.apply(safety_likelihood, axis=1)

# Identify numeric and categorical columns
numeric_features = ['age_numeric', 'sample_size', 'sample_size_log', 'safety_likelihood']
categorical_features = ['gender', 'location']

# Split features and target
X = data[numeric_features + categorical_features]
y = data['likelihood_percent']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the full pipeline
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', None)  # This will be set in the loop
])

# Define models
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'Linear Regression': LinearRegression()
}

# Define parameter grids for GridSearchCV
param_grids = {
    'Random Forest': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [None, 10, 20, 30]
    },
    'SVR': {
        'regressor__C': [0.1, 1, 10],
        'regressor__kernel': ['rbf', 'linear']
    },
    'Linear Regression': {}
}

# Train and evaluate models
for name, model in models.items():
    print(f"\nTraining {name}...")

    # Set the regressor in the pipeline
    full_pipeline.set_params(regressor=model)

    # Perform GridSearchCV
    grid_search = GridSearchCV(full_pipeline, param_grids[name], cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    # Get best model
    best_model = grid_search.best_estimator_

    # Make predictions
    y_pred = best_model.predict(X_test)

    # Evaluate the model
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred))}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
    print(f"R-squared: {r2_score(y_test, y_pred)}")
    print(f"Explained Variance Score: {explained_variance_score(y_test, y_pred)}")
    print(f"Max Error: {max_error(y_test, y_pred)}")

    # Feature importance (only for Random Forest)
    if name == 'Random Forest':
        # Get feature names for numeric features
        numeric_feature_names = numeric_features

        # Get feature names for categorical features after one-hot encoding
        onehot_encoder = best_model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
        cat_feature_names = onehot_encoder.get_feature_names_out(categorical_features).tolist()

        # Combine all feature names
        feature_names = numeric_feature_names + cat_feature_names

        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': best_model.named_steps['regressor'].feature_importances_
        }).sort_values('importance', ascending=False)
        print("\nFeature Importance:")
        print(feature_importance)


# Analyze safety likelihood
print("\nSafety Likelihood Analysis:")
print(f"Average safety likelihood: {data['safety_likelihood'].mean()}")
print(f"Safety likelihood by gender:")
print(data.groupby('gender')['safety_likelihood'].mean())

# Correlation between safety likelihood and smoking likelihood
correlation = data['safety_likelihood'].corr(data['likelihood_percent'])
print(f"\nCorrelation between safety likelihood and smoking likelihood: {correlation}")

# Analyze smoking likelihood by location and gender
print("\nSmoking Likelihood Analysis:")
smoking_by_location_gender = data.groupby(['location', 'gender'])['likelihood_percent'].mean().unstack()
print(smoking_by_location_gender)

# Identify high-risk areas (high smoking likelihood, low safety)
data['risk_score'] = data['likelihood_percent'] * (1 - data['safety_likelihood'])
high_risk_areas = data.groupby('location')['risk_score'].mean().sort_values(ascending=False).head(5)
print("\nTop 5 High-Risk Areas:")
print(high_risk_areas)

# Select the best model (SVR in this case)
best_model = grid_search.best_estimator_  # Make sure this is the GridSearchCV object for the SVR model

# Example of making predictions on new data
new_data = pd.DataFrame({
    'age': ['25-34', '45-54', '65+'],
    'gender': ['Male', 'Female', 'Male'],
    'location': ['CBD', 'Suburban', 'Rural'],
    'sample_size': [100, 150, 200]
})

predictions = make_predictions(best_model, new_data)
new_data['predicted_likelihood'] = predictions

print("\nPredictions for new data:")
print(new_data)

# Identify high-risk areas
high_risk_areas = identify_high_risk_areas(new_data)
print("\nHigh-risk areas:")
print(high_risk_areas)

# Add visualizations #Heat Map
plt.figure(figsize=(12, 8))
sns.heatmap(smoking_by_location_gender, annot=True, cmap='YlOrRd')
plt.title('Smoking Likelihood by Location and Gender')
plt.tight_layout()
plt.show()

# Visualize feature importance #Bar Chart
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Feature Importance in Random Forest Model')
plt.tight_layout()
plt.show()

# Visualize the relationship between safety likelihood and smoking likelihood #scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='safety_likelihood', y='likelihood_percent', hue='gender', data=data)
plt.title('Safety Likelihood vs Smoking Likelihood')
plt.xlabel('Safety Likelihood')
plt.ylabel('Smoking Likelihood (%)')
plt.tight_layout()
plt.show()

# Visualize the distribution of risk scores #histogram
plt.figure(figsize=(10, 6))
sns.histplot(data['risk_score'], kde=True)
plt.title('Distribution of Risk Scores')
plt.xlabel('Risk Score')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Visualize top 5 high-risk areas #bargraph
plt.figure(figsize=(10, 6))
high_risk_areas.plot(kind='bar')
plt.title('Top 5 High-Risk Areas')
plt.xlabel('Location')
plt.ylabel('Risk Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualize model performance comparison
model_names = list(models.keys())
mse_scores = []

for name in model_names:
    pipeline = full_pipeline.set_params(regressor=models[name])
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    mse_scores.append(-scores.mean())

plt.figure(figsize=(10, 6))
sns.barplot(x=model_names, y=mse_scores)
plt.title('Model Performance Comparison')
plt.xlabel('Model')
plt.ylabel('Mean Squared Error (Cross-Validation)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualize predictions for new data
plt.figure(figsize=(10, 6))
sns.barplot(x='location', y='predicted_likelihood', hue='gender', data=new_data)
plt.title('Predicted Smoking Likelihood for New Data')
plt.xlabel('Location')
plt.ylabel('Predicted Likelihood (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Visualize high-risk areas
plt.figure(figsize=(10, 6))
sns.barplot(x='location', y='risk_score', data=high_risk_areas)
plt.title('Risk Scores for High-Risk Areas')
plt.xlabel('Location')
plt.ylabel('Risk Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()